# Data_Genration and Preprocessing

## Importing built-in Functions

In [2]:
import numpy as np
import csv as csv
from random import randint
import re
import torch 
import pandas as pd
from sklearn.model_selection import train_test_split
import math

## Defining Preprocessing Functions

In [5]:
def csv2data(fileStr):
    dataStr = csv.reader(open(fileStr), delimiter='\n', quotechar='|')
    data = []
    for row in dataStr:
        eachRow = ','.join(row);
        rowArr = np.asarray(eachRow.split(','));
        data.append(rowArr[-1])
    data = data[1:];
    
    return data

def csv2images(fileStr):
    dataStr = csv.reader(open(fileStr), delimiter='\n', quotechar='|')
    data = []
    next(dataStr)
    for row in dataStr:
        eachRow = ','.join(row);
        rowArr = list(map(int, eachRow.split(',')));
        data.append(rowArr)
    
    return data


def cleanDict(dictionary):
    newDict = [];
    for word in dictionary:
        if((len(list(word))>=4) and (len(word.split(' '))==1) and re.match("^[a-zA-Z]*$", word) and re.match("^[^j|J|z|Z]*$", word)):
            newDict.append(word);
    
    return newDict        

def getWordList(*arg):
    
    dictionary = arg[0]
    if len(arg) == 1:
        numOfFeatures = 100;
    elif len(arg) == 2:
        numOfFeatures = arg[1];
    elif len(arg) == 3:
        numOfFeatures = arg[1];
        numOfMutations = arg[2];
        
    wordList =[];
    
    for word in dictionary:
        row = [];
        for fea in range(numOfFeatures):
            if len(arg) < 3:
                row.append(genMisspellings(word))
            elif len(arg) == 3:
                row.append(genMisspellings(word,numOfMutations))
                
        row.append(word)
        wordList.append(row)
    
    return wordList

def getSeparatedWordList(*arg):
    
    dictionary = arg[0];
    testPercentage = 0.1;
    truePercenage = 0.2;
        
    if len(arg) == 1:
        numOfFeatures = 100;
    elif len(arg) == 2:
        numOfFeatures = arg[1];
    elif len(arg) == 3:
        numOfFeatures = arg[1];
        numOfMutations = arg[2];
    elif len(arg) == 4:
        numOfFeatures = arg[1];
        numOfMutations = arg[2];
        testPercentage = arg[3];
    elif len(arg) == 5:
        numOfFeatures = arg[1];
        numOfMutations = arg[2];
        testPercentage = arg[3];
        truePercenage = arg[4];
        
    trainWordList =[];
    testWordList = [];
        
    noOfMisspellings = math.floor((1-truePercenage)*numOfFeatures);
    noOfTestSamples = math.floor(testPercentage*numOfFeatures);
    
    for word in dictionary:
        row = [];
        for fea in range(noOfMisspellings):
            if len(arg) < 3:
                row.append(genMisspellings(word))
            elif len(arg) >= 3:
                row.append(genMisspellings(word,numOfMutations))
        
        trainRow, testRow = train_test_split(row,test_size=testPercentage);
        
        trainRow.extend([word]*math.ceil((numOfFeatures-noOfMisspellings)*(1-testPercentage)));
        testRow.extend([word]*math.ceil((numOfFeatures-noOfMisspellings)*(testPercentage)));       
        
        trainWordList.append(trainRow)
        testWordList.append(testRow)
        
    return trainWordList,testWordList


def genMisspellings(*arg):
    
    word = arg[0]
    minMut = 0;
    if len(arg) == 1:        
        maxMut = randint(1,int(len(word)/2));
    else:
        maxMut = arg[1]
    
    alphabetArr = ['a','b','c','d','e','f','g','h','i','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y'];
    charArr = list(word);
    
    for mut in range(minMut,maxMut):        
        mutIdx = randint(0,len(word)-1);
        charArr[mutIdx] = alphabetArr[randint(0,23)];    

    return "".join(charArr)

def createBuckets(trainImages):
    buckets = [[] for _ in range(25)];

    for idx, sample in enumerate(trainImages):
        buckets[int(sample[0])].append(idx);
        
    return buckets

def generateData(wordList,buckets):
    
    data = [];
    for sample in wordList:
        label = sample[-1];
        for word in sample:
            refArr = [ord(char) - 96 for char in word.lower()];
            idxArr = [];
            for ele in refArr:
                idxArr.append(buckets[ele-1][randint(0,len(buckets[ele-1])-1)]);
            data.append(list([word,idxArr,label]));    

    return data

def generateTrueData(wordList,buckets):
    
    data = [];
    for word in wordList:       
        refArr = [ord(char) - 96 for char in word.lower()];
        idxArr = [];
        for ele in refArr:
            idxArr.append(buckets[ele-1][randint(0,len(buckets[ele-1])-1)]);
        data.append(list([word,idxArr]));    

    return data

def generateLSTMData(wordList,buckets):
    
    data = [];
    for sample in wordList:
        label = sample[-1];
        labelArr = [ord(char) - 96 for char in label.lower()]
        for word in sample:
            refArr = [ord(char) - 96 for char in word.lower()]
            idxArr = []
            for ele in refArr:
                idxArr.append(buckets[ele-1][randint(0,len(buckets[ele-1])-1)]);
            data.append((word,refArr,label,labelArr));    

    return data

def Gen(listOfwords, trainImages):
    
    # In[13]:
    
    # create buckets containing the indices of the corresponding alphabets
    buckets = createBuckets(trainImages);
    
    # remove words with 'j,J,z,Z' or any special characters
    dictionary = cleanDict(listOfwords);
    
    # generate misspelled words
    # getData(dictionary,numOfFeatures,numOfMutations)
    # OR
    # getData(dictionary)
    wordList = getWordList(dictionary,10,1)
    
    # generate data with misspelled word, index of location of each of the alphabet of the incorrect word and the correct word
    data = generateData(wordList,buckets);
    
    # generate data with word, index of location of each of the alphabet of the word
    #data = generateTrueData(dictionary,buckets);

    return data

def GenSplitData(listOfwords, trainImages, testImages, numOfFeatures, numOfMutations, testPercentage, truePercentage):
    
    # In[13]:
    
    # create buckets containing the indices of the corresponding alphabets
    trainBuckets = createBuckets(trainImages);
    testBuckets = createBuckets(testImages);
    
    # remove words with 'j,J,z,Z' or any special characters
    dictionary = cleanDict(listOfwords);
    
    # generate misspelled words
    # getData(dictionary,numOfFeatures,numOfMutations)
    # OR
    # getData(dictionary)
    # OR
    # getSeparatedWordList(dictionary,numOfFeatures,numOfMutations,testPercentage,truePercentage)
    trainWordList, testWordList = getSeparatedWordList(dictionary, numOfFeatures, numOfMutations, testPercentage, truePercentage)
    
    # generate data with misspelled word, index of location of each of the alphabet of the incorrect word and the correct word
    trainData = generateData(trainWordList,trainBuckets);
    testData = generateData(testWordList,testBuckets);
    
    # generate data with word, index of location of each of the alphabet of the word
    #data = generateTrueData(dictionary,buckets);

    return trainData, testData

def split_indexes(full_data):
    
    item_1=[item[1] for item in full_data]
    idxs=([torch.LongTensor(xi) for xi in item_1])
    item_3=[item[3] for item in full_data]
    labels=([torch.LongTensor(xi) for xi in item_3])
    
    return idxs, labels

In [7]:
listOfwords = csv2data('Misspelled_correct_words_data.csv')

trainImages = csv2images('sign_mnist_train.csv')

testImages = csv2images('sign_mnist_test.csv')


## Genrate Data for LSTM_Base

In [12]:
dictionary = cleanDict(listOfwords);
trainWordList, testWordList = getSeparatedWordList(dictionary, 200, 1, 0.1, 1.0)

trainBuckets = createBuckets(trainImages)
testBuckets = createBuckets(testImages)

trainData = generateLSTMData(trainWordList,trainBuckets)
testData = generateLSTMData(testWordList,testBuckets)

train_index, train_labels =split_indexes(trainData)
test_index, test_labels=split_indexes(testData)


In [19]:
torch.save(train_index,'train_data_20_mut_2_c.pt')
torch.save(train_labels,'train_labels_20_mut_2_c.pt')
torch.save(test_index,'test_data_20_mut_2_c.pt')
torch.save(test_labels,'test_labels_20_mut_2_c.pt')


In [4]:
# create buckets containing the indices of the corresponding alphabets
buckets = createBuckets(trainImages);

# remove words with 'j,J,z,Z' or any special characters
dictionary = cleanDict(listOfwords);

# generate misspelled words
# getData(dictionary,numOfFeatures,numOfMutations)
# OR
# getData(dictionary)
wordList = getWordList(dictionary,100,1)


# generate data with misspelled word, index of location of each of the alphabet of the incorrect word and the correct word
#data = generateData(wordList,buckets);

# generate data with word, index of location of each of the alphabet of the word
#data = generateTrueData(dictionary,buckets);

# getSeparatedWordList(dictionary,numOfFeatures,numOfMutations,testPercentage,truePercentage)
#trainwordList, testwordList = getSeparatedWordList(dictionary, 20, 2, 0.1, 0.3)

# generate data with misspelled word, index of location of each of the alphabet of the incorrect word and the correct word
data = generateLSTMData(wordList,buckets);


##  Generate Data for END-End Model 

In [ ]:

word_to_train_imgs = Gen(listOfwords, trainImages)
word_to_test_imgs = Gen(listOfwords, testImages)

#### now we split both list in equal proportions and same indices...so each proportion contains same words but 
####   different images (train or test imgs) 
trainWordstrainImages, testWordstrainImages, trainWordstestImages, testWordstestImages = train_test_split(word_to_train_imgs, word_to_test_imgs, test_size = 0.30, random_state = 101)

torch.save(trainWordstrainImages, 'trainWordstrainImages.pt')  #### only this is used to train CLSTM, others to be used for testing all models
torch.save(testWordstrainImages, 'testWordstrainImages.pt')
torch.save(trainWordstestImages, 'trainWordstestImages.pt')
torch.save(testWordstestImages, 'testWordstestImages.pt')


In [8]:
# word_to_train_imgs, word_to_test_imgs = GenSplitData(listOfwords, trainImages, testImages, numOfFeatures,numOfMutations,testPercentage,truePercentage)
word_to_train_imgs, word_to_test_imgs = GenSplitData(listOfwords, trainImages, testImages, 102, 1, 0.99, 1.0)

#### now we split both list in equal proportions and same indices...so each proportion contains same words but 
####   different images (train or test imgs) 
#trainWordstrainImages, testWordstrainImages, trainWordstestImages, testWordstestImages = train_test_split(word_to_train_imgs, word_to_test_imgs, test_size = 0.30, random_state = 101)

torch.save(word_to_train_imgs, 'trainWordstrainImages.pt')  #### only this is used to train CLSTM, others to be used for testing all models
#torch.save(testWordstrainImages, 'testWordstrainImages.pt')
#torch.save(trainWordstestImages, 'trainWordstestImages.pt')
torch.save(word_to_test_imgs, 'testWordstestImages.pt')



### Data Preprocessing 

####  Torch Padding

In [ ]:
# padding the data 
misspelled_index_pad=pad_sequence(misspelled_index)
mis_word_pad=torch.transpose(mis_word_pad,1,0)
# padding the labels
correct_word_pad=pad_sequence(correct_index)
correct_word_pad=torch.transpose(correct_word_pad,1,0)

#### Torch Splitting to Train and Test

In [ ]:
# Dividing into test sets and training sets 10 % percent for test or evaluating set
# taking ten percent of data as atest set
train_size = int(0.9 * len(mis_word_pad))
test_size = len(mis_word_pad) - train_size
train, test = torch.utils.data.random_split(mis_word_pad, [train_size, test_size])
train_lbl, test_lbl   = random_split(correct_word_pad,[train_size, test_size])

# splitting with random indices 
train_data=train.dataset[train.indices]
test_data=test.dataset[test.indices]
train_labels=train_lbl.dataset[train.indices]
test_labels=test_lbl.dataset[test.indices]
# saving data 

###  Saving the Data diffrent Formats

#### DataFrame Format

In [85]:
# saving as a data frame 
df = pd.DataFrame(data, columns=['Mis_word', 'Mis_idx', 'correc_word','correc_idx']) 
df.to_csv('LSTM_data.csv', index=False, encoding='utf-8')  

#### Torch  Version

In [11]:
torch.save(misspelled_words, 'misspelled_words.pt')
torch.save(misspelled_index, 'misspelled_index.pt')
torch.save(correct_words, 'correct_words.pt')
torch.save(correct_index, 'correct_index.pt')